In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from diplotype import Diplotype


# Load the caller outputs pickle file

In [2]:
caller_outputs_df = pd.read_parquet("output/1_caller_outputs.parquet")


# Convert genotype calls to diplotypes

In [3]:
def get_diplotype(row: pd.Series) -> Diplotype:
    gt = row["genotype"]
    filt = row["filter"]
    novel = row["novel_allele"]
    if gt is None:
        return Diplotype.no_call()
    elif "*other" in gt or "/" not in gt or "or" in gt:
        return Diplotype.invalid(gt)
    elif novel:
        return Diplotype.novel_allele(raw_diplotype=gt, filt=filt)
    else:
        return Diplotype.from_string(raw_diplotype=gt, filt=filt)
    

In [4]:
caller_outputs_df["diplotype"] = caller_outputs_df.apply(get_diplotype, axis=1)
caller_outputs_df["parsed_diplotype"] = caller_outputs_df["diplotype"].apply(
    lambda x: x.parsed
)
caller_outputs_df["parsed_haplotypes"] = caller_outputs_df["diplotype"].apply(
    lambda x: [haplotype.parsed for haplotype in x.haplotypes]
)
caller_outputs_df["parsed_star_alleles"] = caller_outputs_df["diplotype"].apply(
    lambda x: [star_allele.parsed for star_allele in x.star_alleles]
)
caller_outputs_df.head(5)


,sample_id,genotype,filter,copy_number,caller,novel_allele,diplotype,parsed_diplotype,parsed_haplotypes,parsed_star_alleles
0,WHB3374,*1/*10,None,2.0,aldy,False,"Diplotype(parsed='*1/*10', filt=None, is_novel...",*1/*10,"[*1, *10]","[*1, *10]"
1,WHB3374,*1/*10,PASS,4.0,cyrius,False,"Diplotype(parsed='*1/*10', filt='PASS', is_nov...",*1/*10,"[*1, *10]","[*1, *10]"
2,WHB3374,*1/*10,None,2.0,stellarpgx,False,"Diplotype(parsed='*1/*10', filt=None, is_novel...",*1/*10,"[*1, *10]","[*1, *10]"
3,WHB3375,*10/*36.ALDY,None,2.0,aldy,False,"Diplotype(parsed='*10/*36', filt=None, is_nove...",*10/*36,"[*10, *36]","[*10, *36]"
4,WHB3375,*5/*36+*10,PASS,4.0,cyrius,False,"Diplotype(parsed='*5/*10+*36', filt='PASS', is...",*5/*10+*36,"[*5, *10+*36]","[*5, *36, *10]"


# Consensus

In [5]:
def get_diplotype_consensus(sample_df: pd.DataFrame) -> dict:
    counts = sample_df["parsed_diplotype"].value_counts(normalize=True, dropna=False)

    consensus = None
    if counts.max() > 0.5:
        consensus = list(counts.items())[0][0]

    callers_in_agreement = list(
        sample_df.apply(
            lambda x: x["caller"] if x["parsed_diplotype"] == consensus else np.nan,
            axis=1,
        ).dropna()
    )

    return {
        "diplotype_consensus": consensus,
        "diplotype_caller_agreement": counts.max() if counts.max() > 0.5 else None,
        "callers_in_agreement": callers_in_agreement,
        "novel_allele": list(sample_df[sample_df["novel_allele"] == True]["caller"]),
        # "diplotype_counts": list(zip(counts.index.values, counts.values)),
    }


def get_haplotype_consensus(sample_df: pd.DataFrame) -> dict:
    counts_by_pos = {}
    haplotypes = sample_df["parsed_haplotypes"]
    max_length = haplotypes.apply(len).max()
    for i in range(max_length):
        mode = haplotypes.apply(lambda x: x[i] if len(x) > i else None).mode(
            dropna=False
        )
        if len(mode) < max_length:
            counts_by_pos[i] = mode[0]
        else:
            counts_by_pos[i] = None
    return {"haplotype_consensus": pd.Series(counts_by_pos).to_list()}


def get_consensus(sample_df: pd.DataFrame) -> pd.Series:
    original_calls = {}

    for caller in sample_df["caller"].unique():
        genotypes = sample_df.query("caller == @caller")["genotype"]
        original_calls[caller] = list(genotypes)

    stellarpgx_copy_number = sample_df.query("caller == 'stellarpgx'").iloc[0][
        "copy_number"
    ]

    diplotype_consensus = get_diplotype_consensus(sample_df)
    haplotype_consensus = get_haplotype_consensus(sample_df)

    return pd.Series(
        {
            "stellarpgx_copy_number": stellarpgx_copy_number,
            **original_calls,
            **diplotype_consensus,
            **haplotype_consensus,
        }
    )


In [6]:
consensus_df = caller_outputs_df.groupby("sample_id").apply(get_consensus)
# consensus_df.sample(10)


/tmp/ipykernel_449302/1279046289.py:29: UserWarning: Unable to sort modes: '<' not supported between instances of 'NoneType' and 'str'
  mode = haplotypes.apply(lambda x: x[i] if len(x) > i else None).mode(
/tmp/ipykernel_449302/1279046289.py:29: UserWarning: Unable to sort modes: '<' not supported between instances of 'NoneType' and 'str'
  mode = haplotypes.apply(lambda x: x[i] if len(x) > i else None).mode(
/tmp/ipykernel_449302/1279046289.py:29: UserWarning: Unable to sort modes: '<' not supported between instances of 'NoneType' and 'str'
  mode = haplotypes.apply(lambda x: x[i] if len(x) > i else None).mode(
/tmp/ipykernel_449302/1279046289.py:29: UserWarning: Unable to sort modes: '<' not supported between instances of 'NoneType' and 'str'
  mode = haplotypes.apply(lambda x: x[i] if len(x) > i else None).mode(
/tmp/ipykernel_449302/1279046289.py:29: UserWarning: Unable to sort modes: '<' not supported between instances of 'NoneType' and 'str'
  mode = haplotypes.apply(lambda x: x

# Add sample metadata

In [7]:
metadata_df = pd.read_csv(
    "/home/jupyter-yusuf/aws-s3-cyp2d6/SG10K_DRAGEN_CYP2D6_sample_metadata.csv"
)
metadata_df = metadata_df.set_index("npm_research_id")
consensus_df = consensus_df.merge(
    metadata_df.loc[:, ["genetic_sex", "genetic_ancestry"]],
    how="left",
    left_index=True,
    right_index=True,
)


## Remove Others and Potential Novel Alleles

In [8]:
consensus_df = consensus_df[consensus_df["genetic_ancestry"] != "O"]


In [9]:
len(consensus_df)

1850

# Export to TSV and parquet

In [10]:
output_dir = Path("output")
file_name = "2_consensus"

if not output_dir.exists():
    output_dir.mkdir()

consensus_df = consensus_df.reset_index()

consensus_df.to_csv(output_dir / f"{file_name}.tsv", index=False, sep="\t")
consensus_df.to_parquet(output_dir / f"{file_name}.parquet", index=False)


In [21]:
len(consensus_df) - consensus_df['diplotype_caller_agreement'].isna().sum()
# consensus_df.head()
# len(consensus_df)

1504

#  Curation of the novel alleles and no consensus diplotype

In [12]:
no_consensus_df = consensus_df[consensus_df['diplotype_consensus'].isna()]

len(no_consensus_df[no_consensus_df.index.isin(potential_novel_alleles)]), len(potential_novel_alleles)

(12, 28)

In [14]:
mask_aldy = no_consensus_df['aldy'].apply(lambda x: ('+' in x[0] or 'x' in x[0]) if x and x[0] is not None else False)
mask_cyrius = no_consensus_df['cyrius'].apply(lambda x: ('+' in x[0] or 'x' in x[0]) if x and x[0] is not None else False)
mask_stellarpgx = no_consensus_df['stellarpgx'].apply(lambda x: ('+' in x[0] or 'x' in x[0]) if x and x[0] is not None else False)

combined_mask = ~(mask_aldy | mask_cyrius | mask_stellarpgx)

filtered_df = no_consensus_df[combined_mask]
len(filtered_df)

26